In [ ]:
import sqlite3
import os
import pandas as pd

# Google Sheet ID (from the URL)
SHEET_ID = "158mz7xAjDFkdbqp3O21ImE8iuDVOflKmgsl9-1Y5QcE"
SHEET_NAME = "Sheet1"
csv_url = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}"

df = pd.read_csv(csv_url, dtype=str).fillna("-")

# Clean DOI column
def clean_doi(doi):
    return doi.replace("https://doi.org/", "").strip()

# Drop columns that start with "Unnamed"
df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

df["DOI"] = df["DOI"].apply(clean_doi)
df = df.drop_duplicates(subset=["DOI"])  # Remove duplicate DOI entries

db_path = "qubit_data.db"
if os.path.exists(db_path):
    os.remove(db_path)  # Remove existing DB to start fresh

conn = sqlite3.connect(db_path)
c = conn.cursor()

# Create main table
c.execute('''
CREATE TABLE main (
    DOI TEXT PRIMARY KEY,
    Hardware_Type TEXT,
    Year TEXT,
    Number_of_Qubits TEXT,
    Temperature_K TEXT,
    Summary TEXT,
    Notes TEXT,
    Other TEXT
)
''')

# Create tables for normalized data
normalized_tables = {
    "T1_Times": "T1 Times (ms)",
    "T2_Times": "T2 Times (ms)",
    "Single_Qubit_Fidelity": "Single Qubit gate fidelity",
    "Two_Qubit_Fidelity": "Two Qubit Gate Fidelity",
    "Measurement_Error": "Measurement Error",
    "State_Prep_Error": "State Preperation Error",
    "SPAM_Error": "SPAM Error"
}

for table, col in normalized_tables.items():
    c.execute(f'''
    CREATE TABLE {table} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        DOI TEXT,
        Value REAL,
        Uncertainty REAL,
        Notes TEXT,
        FOREIGN KEY (DOI) REFERENCES main(DOI)
    )
    ''')

# Insert data into the main table
for _, row in df.iterrows():
    c.execute('''
    INSERT INTO main (DOI, Hardware_Type, Year, Number_of_Qubits, Temperature_K, Summary, Notes, Other)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row["DOI"].strip(), row["Hardware Type"].strip(), row["Year"].strip(), row["Number of Qubits"].strip(),
          row["Temperature (K)"].strip(), row["Summary"].strip(), row["Notes"].strip(), row["Other"].strip()))

# Function to insert parsed values into normalized tables
def insert_normalized_data(table_name, doi, entry):
    if entry == "-":  # Skip empty values
        return

    values = entry.split(";")  # Split multiple entries
    for value in values:
        value = value.strip()
        if ":" in value:  # Check if uncertainty is present
            num_part, unc_part = value.split(":")
            num_part = num_part.strip()
            unc_part = unc_part.strip()
        else:
            num_part = value
            unc_part = None

        c.execute(f'''
        INSERT INTO {table_name} (DOI, Value, Uncertainty)
        VALUES (?, ?, ?)
        ''', (doi, num_part, unc_part))


def insert_normalized_data_fixed(table_name, doi, entry):
    if entry == "-":
        return
    values = entry.split(";")
    for value in values:
        value = value.strip()
        num_part, unc_part = None, None
        if ":" in value:
            parts = value.split(":")
            num_part = parts[0].strip()
            unc_part = parts[1].strip() if len(parts) > 1 else None
        else:
            num_part = value
        try:
            num_part = float(num_part) if num_part else None
            unc_part = float(unc_part) if unc_part else None
            # print(f"Inserting into {table_name}: DOI={doi}, Value={num_part}, Uncertainty={unc_part}")
            c.execute(f'''
            INSERT INTO {table_name} (DOI, Value, Uncertainty)
            VALUES (?, ?, ?)
            ''', (doi, num_part, unc_part))
        except ValueError:
            print(f"Skipping invalid entry in {table_name}: DOI={doi}, entry={value}")

# Add before the loop
# print("DataFrame columns:", df.columns.tolist())

# Insert data
for table, col in normalized_tables.items():
    print(f"Processing table {table} with column {col}")
    for _, row in df.iterrows():
        # print(f"Row DOI={row['DOI']}, {col}={row[col]}")
        insert_normalized_data_fixed(table, row["DOI"].strip(), row[col])

conn.commit()
conn.close()

print("Database and tables created successfully.")

Processing table T1_Times with column T1 Times (ms)
Processing table T2_Times with column T2 Times (ms)
Processing table Single_Qubit_Fidelity with column Single Qubit gate fidelity
Processing table Two_Qubit_Fidelity with column Two Qubit Gate Fidelity
Processing table Measurement_Error with column Measurement Error
Processing table State_Prep_Error with column State Preperation Error
Processing table SPAM_Error with column SPAM Error
Database and tables created successfully.


In [ ]:
'''
normalized_tables = {
    "T1_Times": "T1 Times (ms)",
    "T2_Times": "T2 Times (ms)",
    "Single_Qubit_Fidelity": "Single Qubit gate fidelity",
    "Two_Qubit_Fidelity": "Two Qubit Gate Fidelity",
    "Measurement_Error": "Measurement Error",
    "State_Prep_Error": "State Preperation Error",
    "SPAM_Error": "SPAM Error"
}
'''

# Connect to the SQLite database
db_path = "qubit_data.db"
conn = sqlite3.connect(db_path)

# Query to get T1 times
query = """
SELECT * FROM main
"""

# Read the data into a Pandas DataFrame
df = pd.read_sql_query(query, conn)
print(df)

                                  DOI           Hardware_Type  Year  \
0           10.1038/s41586-019-1666-5  Superconducting Qubits  2019   
1      10.1103/PhysRevLett.117.060504            Trapped Ions  2016   
2      10.1103/PhysRevLett.117.060505            Trapped Ions  2016   
3      10.1103/PhysRevLett.113.220501            Trapped Ions  2014   
4            10.1088/2058-9565/aca21f  Superconducting Qubits  2023   
..                                ...                     ...   ...   
96   10.1103/PhysRevResearch.2.033447  Superconducting Qubits  2020   
97   10.1103/PhysRevApplied.16.024050  Superconducting Qubits  2021   
98                10.1038/nature01494            Trapped Ions  2003   
99                10.1038/nature02015  Superconducting Qubits  2003   
100        10.1038/s41534-020-00346-2  Superconducting Qubits  2021   

    Number_of_Qubits Temperature_K  \
0                 53          0.02   
1                  2           300   
2                  2           30